## Introduction
#### 데이터셋을 분류하고 각 그룹에 집계나 변형 같은 함수를 적용하는일은 데이터분석 과정의 핵심이다. 
#### pandas는 데이터 집합을 자연스럽게 나누고 요약할 수 있는 groupby라는 유연한 방법을 제공한다. 
#### 이 장에서 배우게 될 내용은 다음과 같다.
#### 1. 하나 이상의 키(함수, 배열, DataFrame의 컬럼이름)를 이용해서 객체를 여러 조각으로 나누는 방법
#### 2. 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약 통계를 계산하는 방법
#### 3. 정규화, 선형회귀, 등급 또는 선택 같은 집단 내 변형이나 다른 조작을 적용하는 방법 
#### 4. 피벗 테이블과 교차 알람표를 구하는 방법
#### 5. 변위치 분석과 다른 통계 집단 분석을 수행하는 방법

## GroupBy 작동방법
#### 분리-적용-결합(split-apply-combine')은 그룹연산을 잘 설명하는 용어이다. 
#### 1. 그룹연산의 첫 번째는 데이터를 하나 이상의 키를 기준으로 분리한다. 예를들어 로우(axis=0) 또는 컬럼(axis=1)로 분리한다.
#### 2. 분리하고 나서 함수를 각 그룹에 적용시켜 새로운 값을 얻어낸다. 
#### 3. 함수를 적용한 결과를 하나의 객체로 결합한다. 
![ex_screenshot](그림10-1.png)

#### 각 그룹의 색인은 다음과 같이 다양한 형태가 될수 있으며, 모두 같은 타입일 필요가 없다 
####     1. 그룹으로 묶을 축과 동일한 길이의 리스트나 배열
####     2. DataFrame의 컬럼 이름을 지칭하는 값
####     3. 그룹으로 묶을 값과 그룹 이름에 대응하는 사전이나 Series 객체
####     4. 축 색인 혹은 색인 내의 개별 이름에 대해 실행되는 함수 
#### 1,2,3번의 세 방법은 객체를 나눌때 사용할 배열을 생성하기 위한 방법이다. 

In [13]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randn(5),
                  'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.092372,-0.316831
1,a,two,-1.064640,-0.339453
2,b,one,1.615877,0.425427
3,b,two,-2.470148,0.310790
4,a,one,0.184934,1.847614


#### 위 데이터를 key1으로 묶고 각 그룹에서 data1의 평균을 구한다. 
#### 여러방법이 있지만, data1에 대해 groupby 메서드를 호출하고 key1 컬럼을 넘기는 방법을 사용한다. 

In [14]:
grouped = df['data1'].groupby(df['key1'])  #groubed 변수는 Groupby 객체이다.
grouped
#groubed 변수는 Groupby 객체이다.
#df['key1']에 의해 참조되는 중간값에 대한 것 외에는 아무것도 계산되지 않은 객체이다. 
#이 객체는 그룹 연산을 위해 필요한 모든 정보를 가지고 있으며, 각 그룹에 어떤 연산을 적용할 수 있게 해준다.

In [15]:
grouped.mean() #그룹별 평균을 구하려면 Groupby 객체의 mean 메서드 사용
# 데이터가 그룹색인에 따라 수집된다. 즉, key1 컬럼에 있는 유일한 값으로 색인되는 새로운 Series객체가 생성된다.
# 새로 생성된 객체의 색인은 'key1'인데, 그 이유는 DataFrame 컬럼인 df['key1']이기 때문이다.

key1
a   -0.324026
b   -0.427136
Name: data1, dtype: float64

In [16]:
grouped = df['data1'].groupby(df['key1']).mean()
grouped

key1
a   -0.324026
b   -0.427136
Name: data1, dtype: float64

In [17]:
df.pivot_table(index = 'key1', values = 'data1', aggfunc = 'mean')
#pivot_table이 더 쉬워 보이지만 data의 양이 많을 때는 groupby 써라 연산속도가 더 빠르다.

,data1
key1,
a,-0.324026
b,-0.427136


#### 만일 여러개의 배열을 리스트로 넘겼다면 조금 다른 결과를 얻게 된다. 

In [18]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means  
#데이터를 두개의 색인으로 묶었고, 그 결과 계층적 색인을 가지는 Series 객체를 얻을 수 있다.

key1  key2
a     one     0.046281
      two    -1.064640
b     one     1.615877
      two    -2.470148
Name: data1, dtype: float64

In [19]:
means = df.groupby([df['key1'], df['key2']]).mean()
means

data1     data2
key1 key2                    
a    one   0.046281  0.765392
     two  -1.064640 -0.339453
b    one   1.615877  0.425427
     two  -2.470148  0.310790

In [20]:
df.pivot_table(index = ['key1', 'key2'], values = ['data1', 'data2'], aggfunc = 'mean')

data1     data2
key1 key2                    
a    one   0.046281  0.765392
     two  -1.064640 -0.339453
b    one   1.615877  0.425427
     two  -2.470148  0.310790

In [21]:
means.unstack()

data1               data2          
key2       one       two       one       two
key1                                        
a     0.046281 -1.064640  0.765392 -0.339453
b     1.615877 -2.470148  0.425427  0.310790

In [22]:
# 이 예제에서는 그룹의 색인 모두 Series 객체이다. 길이만 같다면 어떤 배열이라도 상관 없다. 
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -1.064640
            2006    1.615877
Ohio        2005   -1.281260
            2006    0.184934
Name: data1, dtype: float64

#### 한 그룹으로 묶을 정보는 주로 같은 DataFrame 안에서 찾게되는데, 이 경우 컬럼 이름(문자열, 숫자 혹은 다른 객체)을 넘겨서 그룹의 색인으로 사용할 수 있다. 

In [23]:
df

,key1,key2,data1,data2
0,a,one,-0.092372,-0.316831
1,a,two,-1.064640,-0.339453
2,b,one,1.615877,0.425427
3,b,two,-2.470148,0.310790
4,a,one,0.184934,1.847614


In [24]:
df.groupby('key1').mean()
#key2 컬럼이 결과에서 빠졌다. df['key2']는 숫자 데이터가 아니다. 이런 컬럼은 성가신 컬럼(nuisance column)이라고 하며 결과에서 제외한다.

TypeError: agg function failed [how->mean,dtype->object]

In [25]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.046281  0.765392
     two  -1.064640 -0.339453
b    one   1.615877  0.425427
     two  -2.470148  0.310790

In [26]:
df.groupby(['key1', 'key2']).size()
# size 메서드는 그룹의 크기를 담고 있는 Series를 반환한다.

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 그룹 간 순회하기
#### GroupBy 객체는 순회(iteration)을 지원하는데, 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환한다. 

In [27]:
df

,key1,key2,data1,data2
0,a,one,-0.092372,-0.316831
1,a,two,-1.064640,-0.339453
2,b,one,1.615877,0.425427
3,b,two,-2.470148,0.310790
4,a,one,0.184934,1.847614


In [28]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.092372 -0.316831
1    a  two -1.064640 -0.339453
4    a  one  0.184934  1.847614
b
  key1 key2     data1     data2
2    b  one  1.615877  0.425427
3    b  two -2.470148  0.310790


In [29]:
for (k1, k2), group in df.groupby(['key1', 'key2']):  #색인이 여럿 존재하는 경우 튜플의 첫 번째 원소가 색인값이 된다.
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.092372 -0.316831
4    a  one  0.184934  1.847614
('a', 'two')
  key1 key2    data1     data2
1    a  two -1.06464 -0.339453
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.615877  0.425427
('b', 'two')
  key1 key2     data1    data2
3    b  two -2.470148  0.31079


#### 당연히 이 안에서 원하는 데이터만 골라낼 수 있다. 한줄이면 그룹별 데이터를 사전형으로 쉽게 바꾸어 사용가능하다. 

In [30]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one -0.092372 -0.316831
 1    a  two -1.064640 -0.339453
 4    a  one  0.184934  1.847614,
 'b':   key1 key2     data1     data2
 2    b  one  1.615877  0.425427
 3    b  two -2.470148  0.310790}

In [31]:
pieces['b']

,key1,key2,data1,data2
2,b,one,1.615877,0.425427
3,b,two,-2.470148,0.310790


In [32]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [33]:
#groupby메서드는 기본적으로 로우 기준(axis=0)으로 그룹을 만들지만, 다른축으로 만드는 것도 가능하다.
#df의 컬럼을 dtyte에 따라 그룹으로 묶을 수도 있다.
grouped = df.groupby(df.dtypes, axis=1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.092372 -0.316831
1 -1.064640 -0.339453
2  1.615877  0.425427
3 -2.470148  0.310790
4  0.184934  1.847614
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


C:\Users\user\AppData\Local\Temp\ipykernel_8676\3849773227.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby(df.dtypes, axis=1)


### 컬럼이나 컬럼의 일부만 선택하기 

In [34]:
'''
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
'''

"\ndf.groupby('key1')['data1']\ndf.groupby('key1')[['data2']]\n"

#### 위 코드는 아래 코드에에 대한 신택틱 슈가(같은 효과)로 같은 결과를 반환한다. 

In [35]:
'''
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])
'''

"\ndf['data1'].groupby(df['key1'])\ndf[['data2']].groupby(df['key1'])\n"

#### 대용량 데이터를 다룰 경우 소수의 컬럼만 집계하고 싶을때가 종종 있다. 아래와 같이 작성한다. 

In [37]:
df.groupby(['key1', 'key2'])[['data2']].mean() #data2 컬럼에 대한 평균만 구하고 싶을때
#df.groupby(['key1', 'key2'])['data2'].mean() #data2 컬럼에 대한 평균만 구하고 싶을때

data2
key1 key2          
a    one   0.765392
     two  -0.339453
b    one   0.425427
     two   0.310790

In [38]:
df.groupby(['key1', 'key2'])['data2'].mean() #series 형태

key1  key2
a     one     0.765392
      two    -0.339453
b     one     0.425427
      two     0.310790
Name: data2, dtype: float64

In [39]:
s_grouped = df.groupby(['key1', 'key2'])['data2']  #groupby 메서드에 리스트나 배열을 넘겼을때는 Groupby객체가 된다.
s_grouped

In [40]:
s_grouped.mean()

key1  key2
a     one     0.765392
      two    -0.339453
b     one     0.425427
      two     0.310790
Name: data2, dtype: float64

In [43]:
s_grouped = df.groupby(['key1', 'key2'])[['data2']]  #groupby 메서드에 리스트나 배열을 넘겼을때는 Groupby객체가 된다.
s_grouped

In [44]:
s_grouped.mean()

data2
key1 key2          
a    one   0.765392
     two  -0.339453
b    one   0.425427
     two   0.310790

### 사전과 Series에서 그룹핑 하기 

In [48]:
people = pd.DataFrame(np.random.randn(5,5),
                     columns = ['a', 'b', 'c', 'd', 'e'],
                     index = ['Joe', 'steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1,2]] = np.nan # a few NA values

people

,a,b,c,d,e
Joe,0.120257,2.468883,0.565111,0.289601,-1.792181
steve,-0.233314,1.024599,-0.104109,1.958351,-0.315497
Wes,0.902886,NaN,NaN,0.330505,-0.293369
Jim,-0.802918,0.465972,-1.090585,1.998810,-1.556599
Travis,2.379336,-0.071399,-1.715113,1.469602,-0.478783


In [49]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

In [50]:
by_column = people.groupby(mapping, axis=1)  #f는 사용되지 않음. 사용하지 않는 그룹 키도 문제 없다는것을 보여주는 예
by_column.sum()

#by_column = people.groupby(mapping, axis=1).sum()
#by_column

C:\Users\user\AppData\Local\Temp\ipykernel_8676\2189012987.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis=1)  #f는 사용되지 않음. 사용하지 않는 그룹 키도 문제 없다는것을 보여주는 예


,blue,red
Joe,0.854711,0.796959
steve,1.854241,0.475788
Wes,0.330505,0.609517
Jim,0.908225,-1.893545
Travis,-0.245511,1.829154


In [52]:
map_series = pd.Series(mapping)

map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [53]:
people.groupby(map_series, axis=1).count()

C:\Users\user\AppData\Local\Temp\ipykernel_8676\1833935060.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis=1).count()


,blue,red
Joe,2,3
steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 함수로 그룹핑 하기 

In [54]:
people

,a,b,c,d,e
Joe,0.120257,2.468883,0.565111,0.289601,-1.792181
steve,-0.233314,1.024599,-0.104109,1.958351,-0.315497
Wes,0.902886,NaN,NaN,0.330505,-0.293369
Jim,-0.802918,0.465972,-1.090585,1.998810,-1.556599
Travis,2.379336,-0.071399,-1.715113,1.469602,-0.478783


In [55]:
people.groupby(len).sum() #배열을 넘기는 대신 len 함수를 넘긴 경우
# people은 사람의 이름을 색인값으로 사용하였다. len 함수를 넘기면 이름의 길이별로 그룹을 묶어준다.
#people.groupby(type).sum() #배열을 넘기는 대신 len 함수를 넘긴 경우

,a,b,c,d,e
3,0.220225,2.934855,-0.525474,2.618916,-3.642149
5,-0.233314,1.024599,-0.104109,1.958351,-0.315497
6,2.379336,-0.071399,-1.715113,1.469602,-0.478783


In [58]:
people.groupby(type).sum()

,a,b,c,d,e
<class 'str'>,2.366247,3.888056,-2.344696,6.046869,-4.436429


In [56]:
key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()  #내부적으로 모두 배열로 변환대므로 함수를 배열, 사전, Series와 섞어 쓰더러도 문제가 없다.

a         b         c         d         e
3 one  0.120257  2.468883  0.565111  0.289601 -1.792181
  two -0.802918  0.465972 -1.090585  1.998810 -1.556599
5 one -0.233314  1.024599 -0.104109  1.958351 -0.315497
6 two  2.379336 -0.071399 -1.715113  1.469602 -0.478783

In [57]:
people

,a,b,c,d,e
Joe,0.120257,2.468883,0.565111,0.289601,-1.792181
steve,-0.233314,1.024599,-0.104109,1.958351,-0.315497
Wes,0.902886,NaN,NaN,0.330505,-0.293369
Jim,-0.802918,0.465972,-1.090585,1.998810,-1.556599
Travis,2.379336,-0.071399,-1.715113,1.469602,-0.478783


In [59]:
people.groupby([key_list]).sum()

,a,b,c,d,e
one,0.789828,3.493483,0.461002,2.578456,-2.401047
two,1.576418,0.394573,-2.805698,3.468413,-2.035382


### 색인 단계로 그룹핑 하기 
#### 계층적으로 색인된 데이터는 축 색인의 단계 중 하나를 사용해서 편리하게 집계할 수 있다. 

In [76]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                   names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.132882 -0.119254  0.691554 -0.655330 -0.833415
1      0.466024 -0.166835 -1.700780 -1.520376  0.837812
2     -0.628400  0.855893  0.172472 -1.047501 -0.483089
3      0.137910 -0.288526  0.910494 -2.007358  1.650113

In [61]:
hier_df.groupby(level='cty', axis=1).count()

C:\Users\user\AppData\Local\Temp\ipykernel_8676\2333815484.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level='cty', axis=1).count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [62]:
hier_df.groupby(level='tenor', axis=1).count()

C:\Users\user\AppData\Local\Temp\ipykernel_8676\2106149719.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level='tenor', axis=1).count()


tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


In [77]:
#5월 8일

### 데이터 집계
#### 데이터 집계는 배열로부터 스칼라 값을 만들어 내는 모든 데이터 변환을 의미한다. 
#### count, sum, mean, median, std, var, min, max, prod, first, last
#### 직점 만들어낸 함수로 사용하는것도 가능하고, 객체에서 이미 정의된 메서드를 사용하는 것도 가능하다.
#### 예를들어 'quantile'메서드는 Series나 DataFrame 객체의 컬럼의 위치를 계산하는 메서드이다.

In [63]:
df

,key1,key2,data1,data2
0,a,one,-0.092372,-0.316831
1,a,two,-1.064640,-0.339453
2,b,one,1.615877,0.425427
3,b,two,-2.470148,0.310790
4,a,one,0.184934,1.847614


In [64]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)
#quantile() 의 괄호 안에 0~1 사이의 값을 넣어주면 분위수를 계산해주며, 
#최소값과 최대값을 등분하여 그 사이를 interpolation 하여 분위수를 계산

key1
a    0.129473
b    1.207274
Name: data1, dtype: float64

#### 자신만의 데이터 집계함수를 사용하려면 배열의 aggregate나 agg 메서드에 해당 함수를 넘기면 된다 

In [65]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
grouped.describe()

### 컬럼에 여러가지 함수 적용하기 

In [66]:
tips = pd.read_csv('tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill'] #tip 데이터에 팀의 비율을 담기 위한 tip_pct를 추가

tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


#### 모든 컬럼을 집계하는것은 mean이나 std등의 메서드를 사용하거나, 원하는 함수에 aggregate를 사용하는 것이다.
#### 하지만, 컬럼에 따라 다른 함수를 사용하거나 여러개의 함수를 한꺼번에 적용하기 원한다면 아래의 예제로 가능하다. 

In [67]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [68]:
grouped_pct.agg(['mean', 'std', peak_to_peak])
#함수 목록이나 함수 이름을 넘기면 함수 이름을 컬럼으로 하는 DataFrame을 얻게 된다.

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

#### Groupby 객체에서 자동으로 지정하는 컬럼이름을 그대로 쓰지 않아도 된다. 
#### lambda : functions have the name "< lambda >" which is difficult to identify (you can check the function's __name__ attribute).
#### 이름과 함수가 담긴 튜플 리스트를 넘기면, 각 튜플의 첫번째 요소가 DataFrame의 컬럼이름으로사용된다.                                      

In [69]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])
#grouped_pct.agg([('foo', 'mean'), ('bar', 'std')])

C:\Users\user\AppData\Local\Temp\ipykernel_8676\2419490817.py:1: FutureWarning: The provided callable <function std at 0x0000018234D158B0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])


foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

#### DataFrame은 컬럼마다 다른 함수를 적용하거나 여러개의 함수를 모든 컬럼에 적용할 수 있다. 
#### tip_pct와 total_bill 컬럼에 동일한 세가지 통계를 계산한다고 가정하자. 

In [70]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

In [71]:
result['tip_pct']

NameError: name 'result' is not defined

#### 위의 예처럼 컬럼이름과 메서드가 담긴 튜플의 리스트를 넘기는 것도 가능하다. 

In [72]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

grouped['tip_pct', 'total_bill'].agg(ftuples)

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

In [73]:
grouped.agg({'tip': np.max, 'size': 'sum'}) #컬럼마다 다른 함수를 적용하고 싶다면, 컬럼이름에 대응하는 함수가 들어있는 사전을 넘긴다.

C:\Users\user\AppData\Local\Temp\ipykernel_8676\2914114739.py:1: FutureWarning: The provided callable <function max at 0x0000018234D0EE50> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({'tip': np.max, 'size': 'sum'}) #컬럼마다 다른 함수를 적용하고 싶다면, 컬럼이름에 대응하는 함수가 들어있는 사전을 넘긴다.


tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [74]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
            'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 색인되지 않은 형태로 집계된 데이터 변형하기 
#### as_index=False를 넘겨서 색인되지  않게 한다. 

In [75]:
#tips.groupby(['day', 'smoker']).mean()
tips.groupby(['day', 'smoker'], as_index=False).mean()

TypeError: agg function failed [how->mean,dtype->object]

### Apply: 일반적인 분리-적용-병합
#### 가장 일반적인 Groupby 메서드의 목적은 apply
#### apply메서드는 객체를 여러조각으로 나누고, 전달된 함수를 각 조각에 일괄 적용한 후 이를 다시 합친다. 
![ex_screenshot](./그림10-1.png)

In [ ]:
tips[:5]

In [ ]:
tips.sort_values(by='tip_pct')

In [ ]:
#팁 데이터에서 그룹별 상위 5개의 tip_pct 값을 고른다.
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]


top(tips, n=6)

In [ ]:
tips.groupby('smoker').apply(top)

#### If you want to pass a function to apply with other arguments or keywords, you can pass them after the function.
#### NOTE - Beside basic usage, it requires creativity to use apply and get the most out of it. What happens inside a function depends on you, as long as the function returns a pandas object or scalar value.

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')
#tips.groupby(['smoker']).apply(top, n=1, column='total_bill')

#### Inside groupby, when you want to invoke a method, there is a shortcut for it.

In [ ]:
# Normal route
result = tips.groupby('smoker')['tip_pct'].describe()

result

In [ ]:
result.unstack('smoker')

In [ ]:
grouped

In [ ]:
# Shortcut

f = lambda x: x.describe()
grouped.apply(f)

### 그룹 색인 생략하기
#### 앞의 예제에서 반환된 객체는 원본 객체의 각 조각에 대한 색인과 그룹키가 계층적으로 사용되었다.
#### 이런 결과는 groupby 메서드에 group_key=False를 넘겨서 막을 수 있다. 

In [ ]:
tips.groupby('smoker').apply(top)

In [ ]:
tips.groupby('smoker', group_keys=False).apply(top)

### 변위치(Quantile) 및 버킷(Bucket) 분석
#### 8장에서 사용한 cut과 qcut 메서드를 groupby와 조합 사용

In [ ]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                     'data2': np.random.randn(1000)})
frame

In [ ]:
quartiles = pd.cut(frame.data1, 4) #임의의 데이터 묶음을 cut을 사용하여 등간격 구간으로 나눔
quartiles[:10]

#### cut에서 반환된 categorical 객체는 바로 groupby로 넘길 수 있다.

In [ ]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
           'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)
#grouped.apply(get_stats)
grouped.apply(get_stats).unstack()

In [ ]:
grouping = pd.qcut(frame.data1, 10) #qcut을 사용하여 등간격이 아닌 크가가 같은 버킷을 계산
grouping

In [ ]:
grouping = pd.qcut(frame.data1, 10, labels=False) #qcut을 사용하여 등간격이 아닌 크가가 같은 버킷을 계산하고,
grouping                                          #label=False로 변위치 숫자를 구했다.

In [ ]:
grouped = frame.data2.groupby(grouping)

grouped.apply(get_stats).unstack()

### 예제 : 그룹에 따른 값으로 결측치 채우기
#### 누락된 데이터를 정리할때 : dropna 사용
#### 하지만, 누락된 값을 고정된 값이나 혹은 데이터로부터 도출된 값으로 채우고 싶을때 : fillna 

In [ ]:
import numpy as np
import pandas as pd

s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

In [ ]:
s.fillna(s.mean())

In [ ]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
         'Oregon', 'Nevada', 'California', 'Idaho']

# Create 4 copies of elements in both East and West as key
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)

data

In [ ]:
# setting some values to NA
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

In [ ]:
group_key

In [ ]:
data.groupby(group_key).mean()

In [ ]:
# setting the NA values with group means
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

#### 그룹에 따라 미리 정의된 다른 값을 채워넣어야 하는경우 : 각 그룹은 name속성을 가지고 있으므로 이를 이용 

In [ ]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

### 예제 : Random sampling and Permutation
#### 대용량의 데이터에서 랜덤 표본을 뽑아내고자 한다. 
#### Series의 sample 메서드를 사용 

In [ ]:
suits = ['H', 'S', 'C', 'D'] #하트, 스페이드, 클럽, 다이아몬드
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
base_names

In [ ]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []

for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)

In [ ]:
deck[:13]

In [ ]:
def draw(deck, n=5):   #5장의 카드를 랜덤으로 뽑기위해 
    return deck.sample(n)

draw(deck)

In [ ]:
cards

In [ ]:
# Get last letter for suit
get_suit = lambda card: card[-1]   #각 세트별로 2장을 무작위로 샘플링 한다. 세트는 각 카드이름의 마지막 글자이므로
                                   #이를 이용해 그룹을 나누고 apply사용  
deck.groupby(get_suit).apply(draw, n=2)

In [ ]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2) #색인하지 않고 각 그룹별로 무작위로 2장 추출

### 예제: 그룹가중평균(Group Weighted Average)과 상관(Correlation)

In [ ]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                               'b', 'b', 'b', 'b'],
                  'data': np.random.randn(8),
                  'weights': np.random.randn(8)})
df

In [ ]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights']) #category별 가중 편균

grouped.apply(get_wavg)

#### 좀 더 복잡한 예제로 '야후 파이낸스'에서 가져온 주식과 S&P 500(종목코드 SPX) 지수의 종가 데이터를 샇펴본다. 

In [ ]:
close_px = pd.read_csv('stock_px_2.csv', parse_dates = True,
                      index_col = 0)

close_px.info()

In [ ]:
close_px[-4:]

#### 퍼센트 변화율로 일일 수익률을 계산하여 연간 SPX 지수와의 상관을 살펴본다. 

In [ ]:
close_px[0:3]

In [ ]:
spx_corr = lambda x: x.corrwith(x['SPX']) #SPX컬럼과 다른 컬럼과의 상관관계를 계산하는 함수
rets = close_px.pct_change().dropna() #close_px의 퍼센트 변화율을 계산한다. 
rets

In [ ]:
spx_corr = lambda x: x.corrwith(x['SPX']) #SPX컬럼과 다른 컬럼과의 상관관계를 계산하는 함수
rets = close_px.pct_change().dropna() #close_px의 퍼센트 변화율을 계산한다. 
get_year = lambda x: x.year    #각 datatime에서 연도 속성만 반환하는 함수

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

In [ ]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT'])) #개별 두 컬럼나의 상관을 계산하는 것도 가능

### Example: 그룹별 선형회귀 
#### groupby can also be used to perform complex group-wise statistical analysis, as long as the function returns pandas object or scalar value.
#### eg - We can perform Ordinary Least Square (OLS) regression of chunks of data.

In [ ]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

### 피벗 테이블과 교차일람표
#### 'pivot table'은 데이터 요약화 도구이다. 
####  'pivot table'은 데이터를 하나이상의 키로 수집해서 어떤 키는 로우에, 어떤 키는 컬럼에 나열해서 데이터를 정렬한다. 

In [ ]:
tips = pd.read_csv('tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill'] #tip 데이터에 팀의 비율을 담기 위한 tip_pct를 추가
#tips
tips.pivot_table(index=['day', 'smoker']) #day와 smoker집단의 평균(pivot_table의 기본 연산)

#### pivot_table allows you to aggregate by certain columns, group by certaing columns with 'index' parameters, and define columns with 'columns' parameter.
#### We could augment this table to include partial totals by passing 'margins=True'. It will add All row and column labels with corresponding values being group statistics for all data within single tier.
#### The 'All' values are aggregations without taking into account the present groups in the columns or rows.

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns='smoker')

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns='smoker', margins=True)

#### To use a different aggregation function, pass it to 'aggfunc'. eg - 'count' or len will give cross-tabulation (frequencies) of group sizes.
#### If some combinations are empty (NA), you can pass a 'fill_value'.

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', 
                 aggfunc=len, margins=True)

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', 
                 aggfunc='mean', fill_value=0)

### Cross-Tabulations: Crosstab(교차일람표)
#### 교차일람표는 그룹 빈도를 계산하기 위한 피벗 테이블의 특수한 경우이다. 

In [ ]:
d = {0: [1, 'USA', 'Right'],
    1: [2, 'JP', 'Left'],
    2: [3, 'USA', 'Left'],
    3: [4, 'JP', 'Right'],
    4: [5, 'JP', 'Left'],
    5: [6, 'USA', 'Left']}
data = pd.DataFrame.from_dict(d, orient='index',
                              columns=['Sample', 'Nationality', 'Handedness'])
data

In [ ]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

In [ ]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)